# <u> ChatLU Prompt Tuning </u>

This notebook will demonstrate a method to prompt an LLM using Ollama with a SmartSolve algorithmic database and receive both a heuristic and LLM reasoning. We will also benchmark the heuristic results to see if it is a viable alternative to the current decision tree generated by the SmartSolve function.

Before you begin running this code, make sure to **upload the SmartSolve smart database** to this project! In addition, if you want to benchmark LLM results, also **upload a test database in the same format as your SmartSolve database**.

Note: If either database has NaN values, the code will throw an error, so add
filler values for the NaN values before uploading the files. I am working on fixing this issue.

# 0. Download ollama



Running these commands will load ollama to this notebook so that it will run. Note that if you have ollama locally, you will still need to run these commands since google colab can't access your terminal. Ollama documentation can be found here: [ollama.com](https://ollama.com/)

In [ ]:
run(`bash -c "curl https://ollama.ai/install.sh | sh"`)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  64617      0 --:--:-- --:--:-- --:--:-- 64470
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################### 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...


>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Process(`bash -c 'curl https://ollama.ai/install.sh | sh'`, ProcessExited(0))

In [ ]:
# Start Ollama serve in the background
run(`bash -c "ollama serve &"`)

Process(`bash -c 'ollama serve &'`, ProcessExited(0))

In [ ]:
# Pull Mistral model: If you want to load another model, just change the name to the model name (eg. llama3)
run(`ollama pull mistral`)

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIBGBN9YsamGN05W2VzGWTk+8QYlzxoxgfORL/LTD7Huu



2025/04/18 00:52:18 routes.go:1231: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:2048 OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_SCHED_SPREAD:false ROCR_VISIBLE_DEVICES: http_proxy: https_proxy:

[GIN] 2025/04/18 - 00:52:19 | 200 |      63.883µs |       127.0.0.1 | HEAD     "/"


time=2025-04-18T00:52:19.059Z level=INFO source=types.go:130 msg="inference compute" id=GPU-81e8a760-07b4-1905-dde4-dfa8fb337743 library=cuda variant=v12 compute=7.5 driver=12.4 name="Tesla T4" total="14.7 GiB" available="14.6 GiB"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ time=2025-04-18T00:52:19.670Z level=INFO source=download.go:177 msg="downloading ff82381e2bea in 16 257 MB part(s)"
pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏  29 KB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   1% ▕                ▏  34 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   2% ▕                ▏ 102 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   3% ▕     

[GIN] 2025/04/18 - 00:53:03 | 200 | 44.006203018s |       127.0.0.1 | POST     "/api/pull"


pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         
pulling 491dfa501e59... 100% ▕████████████████▏  801 B                         
pulling ed11eda7790d... 100% ▕████████████████▏   30 B                         
pulling 42347cd80dc8... 100% ▕████████████████▏  485 B                         
verifying sha256 digest ⠙ pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         
pulling 491dfa501e59... 100% ▕████████████████▏  801 B                         
pulling ed11eda7790d... 100% ▕████████████████▏   30 B                         
pulling 42347cd80dc8... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


Process(`ollama pull mistral`, ProcessExited(0))

# 1. Perform all necessary imports

Run this if you need to import add these packages to your local julia instance

In [ ]:
] add CSV DataFrames JSON3 Random Statistics StatsBase PromptingTools

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed StructTypes ───── v1.11.0
   Installed JSON3 ─────────── v1.14.2
   Installed OpenAI ────────── v0.9.1
   Installed StreamCallbacks ─ v0.6.1
   Installed PromptingTools ── v0.74.1
    Updating `~/.julia/environments/v1.10/Project.toml`
  [0f8b85d8] + JSON3 v1.14.2
  [670122d1] + PromptingTools v0.74.1
  [2913bbd2] + StatsBase v0.34.4
  [9a3f8284] + Random
  [10745b16] + Statistics v1.10.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [0f8b85d8] + JSON3 v1.14.2
⌅ [e9f21f70] + OpenAI v0.9.1
  [670122d1] + PromptingTools v0.74.1
  [c1b9e933] + StreamCallbacks v0.6.1
  [856f2bd8] + StructTypes v1.11.0
        Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated -m`
Precompiling packages...
   3917.2 ms  ✓ StructTypes
   7751.1 ms  ✓ JSON3
   1340.1 ms  ✓ OpenAI
   4188.8 ms  ✓ 

In [ ]:
using CSV, DataFrames, JSON3, Random, Statistics, StatsBase, PromptingTools

const PT = PromptingTools

┌ Warning: OPENAI_API_KEY variable not set! OpenAI models will not be available - set API key directly via `PromptingTools.OPENAI_API_KEY=<api-key>`!
└ @ PromptingTools ~/.julia/packages/PromptingTools/Xer1u/src/user_preferences.jl:189


PromptingTools

#2. Receive input dataset
We will begin by first getting the dataset input from file. By including the colnames parameter, we allow files with only the data and no feature information to be read.

In [ ]:
# Load the matrix dataset
function load_data(file_path::String; colnames::Union{Nothing, Vector{Symbol}}=nothing)::DataFrame
    if isnothing(colnames)
        return CSV.read(file_path, DataFrame)  # Use headers from file
    else
        return DataFrame(CSV.File(file_path; header=false), colnames)
    end
end

load_data (generic function with 1 method)

# 3. Format input prompt
Format the input smart database into an LLM prompt into Llama 3. Include the test matrices that you wish to know the optimal algorithms for in the prompt.

Still getting an error, trying to debug (heuristics works fine)

In [ ]:
# Format dataset into a concise prompt
function format_test_prompt(data::DataFrame, test_data::DataFrame, algorithms)
    train_rows = eachrow(data)
    test_data_no_alg = select(test_data, Not(:algorithm))

    excluded = Set(["algorithm", "pattern", "time"])
    features = [Symbol(col) for col in names(data) if !(col in excluded)]

    prompt = """
    You are an expert in matrix factorization. Below is a dataset with matrix features (e.g., size, rank, sparsity, etc.)
    and the best algorithm for each matrix.

    Training Dataset Examples:
    """

    # Add few-shot examples
    for row in train_rows
        nt = NamedTuple(row)
        filtered = (; (k => v for (k, v) in nt if k in features)...)
        prompt *= "\nMatrix: $(JSON3.write(filtered)) → Algorithm: $(row.algorithm)"
    end
    # Add test matrix inputs
    prompt *= """

    Now, predict the best algorithm for each of the following $(nrow(test_data)) new matrices.

    ⚠️ Only use one of these algorithms: $(join(algorithms, ", "))
    ⚠️ Do NOT use any other algorithm names (e.g., no variants like "sklumt", "arpack", "clapack")

    Test Matrices:
    """

    for (i, row) in enumerate(eachrow(test_data_no_alg))
        prompt *= "\nMatrix $i: $(JSON3.write(NamedTuple(row)))"
    end

    # Final instruction block
    prompt *= """

    Output only a comma-separated list of the predicted algorithm names in order, one per test matrix.
    Do NOT include matrix numbers, explanations, or extra text.

    ⚠️ Repeat: Output must be a comma-separated list of exactly $(nrow(test_data)) predictions.
    ⚠️ Only use one of: $(join(["\"$a\"" for a in algorithms], ", "))
    ⚠️ No text, no variants, no labels. Just: klu, dgetrf, umfpack, ...
    """

    return prompt
end

format_test_prompt (generic function with 1 method)

# 4. Create LLM call
Use the read function native to Julia to call ollama in order to run Mistral with our input prompt.

In [ ]:
# Query Mistral using run function
function query_mistral(prompt::String)::Union{String, Nothing}
    try
        schema = PT.OllamaSchema() # notice the different schema!
        msg = aigenerate(schema, prompt; model="mistral")
        return strip(msg.content)
    catch e
        println("Error querying LLaMA: $e")
        return nothing
    end
end

query_mistral (generic function with 1 method)

# 5. Evaluate LLM performance
Create a function to handle the pipeline: generate LLM prompt, call Mistral, and use the output to evaluate the performance of the LLM on choosing the optimal algorithm for the test dataset.

In [ ]:
# Evaluate Mistral performance
function evaluate_llm(df::DataFrame, test_data::DataFrame, algorithms)
    true_labels = test_data.algorithm
    prompt = format_test_prompt(df, test_data, algorithms)
    llm_response = query_mistral(prompt)

    if isnothing(llm_response)
        println("Failed to get a response from Mistral.")
        return
    end

    println("Mistral Response:\n", llm_response)

    predicted_labels = split(llm_response, ",")

    if length(predicted_labels) != length(true_labels)
        println("Warning: Mismatch in predictions, adjusting length.")
        predicted_labels = predicted_labels[1:length(true_labels)]
    end

    accuracy = mean(predicted_labels .== true_labels)
    println("Mistral Accuracy: ", round(accuracy * 100, digits=2), "%")
end

evaluate_llm (generic function with 1 method)

# 6. Run Prompting and Benchmarking

Load the files in a dataframe and call the previously defined function to prompt the LLM and evaluate the output. Note that we exclude the error column since it is not used in the LLM prompting.

Note: There are still a bunch of length mismatching errors. For some reason, the LLM is generating more or less predictions than the number of matrices we are passing in. I have a feeling this is because of the way we are passing the data, perhaps there is a better way.

In [ ]:
file_path = "smartdb-lu-no-nan.csv" # Change filename to input data file name
df = load_data(file_path)
select!(df, Not(:error))

test_data = load_data("smartdb-lu-test.csv") # Change filename to test data file name
algorithms = unique(df.algorithm)
evaluate_llm(df, test_data, algorithms)

LoadError: UndefVarError: `train_data` not defined

We can also test the code on some of the database to see how well it works in segmenting the database.

In [ ]:
test_df = df[sample(1:nrow(df), 50, replace = false), :]
evaluate_llm(df, test_df, algorithms)

LoadError: BoundsError: attempt to access Int64 at index [2]

# 7. Generate Heuristic Prompt
We will pivot to evaluate the LLM performance on generating a heuristic. The first function we define will first format the prompt we will send to the LLM to generate a heuristic. For this, we only need the smart input database since no test is required for the prompt.

For readability, we will ask the LLM to make the heuristic a bunch of if/else statements. However, there are also other ways to generate the heuristic that may be more optimal but less readable.

In [ ]:
# Format heuristic prompt
function format_heuristic_prompt(data::DataFrame, algorithms)
    excluded = Set(["algorithm", "pattern", "time"])
    features = [Symbol(col) for col in names(data) if !(col in excluded)]

    full_data = JSON3.write(data)

    feature_list = join(features, ", ")
    feature_gets = join(["    $(s) = get(kwargs, :$(s), missing)" for s in features], "\n")

    prompt = """
    You are an expert in matrix factorization. Below is a dataset with matrix features ($feature_list)
    and the best algorithm $algorithms for each matrix.

    Dataset: $full_data

    Using the dataset provided, please create a heuristic that I can use to find the optimal matrix factorization algorithm
    for any combination of input matrix features. The heuristic must cover all algorithms from the dataset: $(join(algorithms, ", ")).
    Ensure that each algorithm has a well-defined region in the feature space. Do not omit any algorithms or suggest only one algorithm.

    Please follow these steps:
    1. Analyze the dataset to understand how each algorithm performs under different feature conditions.
    2. Identify clear boundaries where one algorithm consistently outperforms others.
    3. Create a heuristic using numerical rules that covers all algorithms.
    4. Provide clear, readable guidelines for selecting the optimal algorithm for any given matrix feature set.
    5. Ensure the heuristic covers all algorithms in the dataset, without suggesting any placeholder or alternative algorithms.

    ### Important Instructions for the Julia Code:

    - You must implement the heuristic as a Julia function using this exact signature:
        ```julia
        function choose_algorithm(; kwargs...)::String
        ```
    - Inside the function, access each matrix feature using the `get` function:
        ```julia
    $feature_gets
        ```
    - Do **not** use named parameters in the function signature (e.g., `; sparsity=...` is not allowed).
    - Use `if/elseif/else` logic to express the heuristic clearly.

    - You must return **only one** of the following algorithms, as a string literal: $algorithms
    - Do **not** modify or extend algorithm names (e.g., avoid returning `"umfpack_triangular"` instead of `"umfpack"`).
    - Each `return` statement must use **only one** of the allowed strings exactly as written above.

    - Note: Some feature names may suggest Boolean values (e.g., `issymmetric`, `ishermitian`, `isreal`), but in the dataset, they are represented as numeric indicators (e.g., 0 or 1). Do **not** treat them as Booleans — instead, compare them numerically using expressions like `issymmetric == 1`.

    - Do **not** use logic like `if issymmetric` — this is incorrect. Always use explicit comparisons like `if issymmetric == 1`.

    Your output should include:
    1. A plain-language explanation of the heuristic rules.
    2. The complete Julia code using the structure above.
    """

    return prompt
end

format_heuristic_prompt (generic function with 1 method)

In [ ]:
function format_heuristic_prompt_casting(data::DataFrame, algorithms)
    features = [Symbol("sparsity")]

    full_data = JSON3.write(data)

    feature_list = ["sparsity"]
    feature_gets = join(["    $(s) = get(kwargs, :$(s), missing)" for s in features], "\n")
    allowed_calls = join(["$a(A)" for a in algorithms], ", ")


    return  """
You are an expert in numerical linear algebra and Julia-based matrix factorization.

Below is a dataset of matrix patterns, where each matrix is annotated with numeric features ($feature_list),
and the best-performing Julia factorization algorithm for that matrix.

Dataset: $full_data

Your task is to write a Julia function that selects the best algorithm based only on the features — including any necessary type casting — and then calls that algorithm on matrix `A`.

### Key Requirements:

1. Use ONLY the following features: $feature_list
   - Do NOT invent or reference any additional features (e.g., `issymmetric`, `istril`, `rank`).
   - You must use only these features when designing the heuristic logic.

2. Do NOT use any outside knowledge or assumptions about feature meanings.
   - For example, do NOT assume that "sparsity > 0.8 means sparse".
   - You must learn all threshold values and conditions from the dataset itself.
   - The meaning of each feature must be entirely data-driven.

3. Do NOT assume the input matrix `A` is sparse or dense.
   - You must infer the required type cast based on the feature values only.
   - Use `Matrix(A)` to convert to dense; use `sparse(A)` to convert to sparse.

4. Algorithm input expectations:
   - `dgetrf` must receive a dense matrix (use `Matrix(A)`)
   - `klu` and `umfpack` must receive sparse matrices (use `sparse(A)` if needed)

5. Use the following Julia function template:

function choose_algorithm_call(; kwargs...)::Any
$feature_gets

    # Heuristic logic based on features

    return dgetrf(Matrix(A))  # or klu(sparse(A)), etc.
end

6. If feature values are missing or NaN, return `nothing`.

7. Your function must return **one** of the following valid calls (with any necessary casting): $allowed_calls
   - The call must be a real function call, not a string.
   - Do NOT use quotes. Do NOT return expressions like `"klu(sparse(A))"`.

### Output Requirements:
1. A plain-language explanation of your heuristic, including what ranges of features lead to what casting and algorithm decisions.
2. The full Julia function `choose_algorithm_call` that returns the actual algorithm call (e.g., `klu(sparse(A))`, not a string). Include comments in the code that explain heuristic choices.

Do not output anything other than the explanation and the function code.
"""
end

format_heuristic_prompt_casting (generic function with 1 method)

# 8. Create the Heuristics
Prompt the LLM and generate the heuristics. Store the resulting Julia code in a new file, heuristic.jl.

In [ ]:
# Create heuristics
function create_heuristics(data::DataFrame, file_name, algorithms)
    prompt = format_heuristic_prompt_casting(data, algorithms)
    response = query_mistral(prompt)

    println(response)

    if isnothing(response)
        println("Failed to get a response from LLaMA.")
        return
    end

    code_match = match(r"```julia\n(.*?)```"s, response)
    if code_match !== nothing
        julia_code = strip(code_match.captures[1]) # Only keeps the julia code
        open(file_name, "w") do f
            write(f, julia_code)
        end
        println("Julia code extracted and saved to heuristic.jl")
    else
        println("No Julia code block found in the output.")
    end
end

create_heuristics (generic function with 1 method)

In [ ]:
file_path = "smartdb-lu-no-nan.csv"
df = load_data(file_path)
select!(df, Not(:error))
algorithms = unique(df.algorithm)

create_heuristics(df, "heuristic.jl", algorithms)

time=2025-04-18T01:02:49.267Z level=WARN source=runner.go:131 msg="truncating input prompt" limit=2048 prompt=24456 keep=5 new=2048


[GIN] 2025/04/18 - 01:03:01 | 200 | 12.565538534s |       127.0.0.1 | POST     "/api/chat"
Based on the provided dataset, it appears that the "sparsity" feature value ranges between 0 (fully dense) and 8 (highly sparse). In this case, we will define a threshold of 3 to separate dense matrices from sparse ones. If the sparsity is less than or equal to 3, the matrix will be considered dense and handled as such. Otherwise, it will be treated as sparse.

Here's the Julia function `choose_algorithm_call` based on this heuristic:

```julia
function choose_algorithm_call(; sparsity = missing)
    if isnan(sparsity) || ismissing(sparsity)
        return nothing
    else
        threshold = 3
        if sparsity <= threshold
            A_type = Matrix(A) # Dense matrix
            return dgetrf(A_type)
        else
            A_type = sparse(A) # Sparse matrix
            # You can choose among 'klu', 'umfpack' based on further analysis of the dataset if needed.
            return klu(A_type)

[ Info: Tokens: 2393 in 12.6 seconds


# 9. Test the Heuristic

In [ ]:
function test_heuristic(test_data, funct)
  correct = 0
  for row in eachrow(test_data)
      features = NamedTuple(row)
      result = funct(; features...)
      ideal_algorithm = row.algorithm
      if ideal_algorithm == result
          correct += 1
      end
  end
  return correct/nrow(test_data)
end

test_heuristic (generic function with 1 method)

Using the heuristic julia code we generated in the previous function call, we will test its accuracy on the test database.

In [ ]:
include("heuristic.jl")

test_df = CSV.read("smartdb-lu-test.csv", DataFrame)
accuracy = test_heuristic(test_df, choose_algorithm)

println("Accuracy: ", accuracy)

Accuracy: 1.0


We can also test the heuristic by taking a partition of the training data.

In [ ]:
include("heuristic.jl")

test_df = df[sample(1:nrow(df), 50, replace = false), :]
accuracy = test_heuristic(test_df, choose_algorithm)

println("Accuracy: ", accuracy)

Accuracy: 0.52


# 10. Application to FFTW Data

The functions discussed so far can also be used to analyze any other dataset as long as you can get the features and list of unique algorithms from the dataset.

Let's go through an example of using an LLM to analyze data for developing an optimal heuristic using an online dataset, the [FFTW dataset](https://www.fftw.org/speed/)! For this example, we will just use the first dataset which performs FFT on 1.06 GHz PowerPC 7447A, MacOSX.

Unfortunately, we can't do what we did previously since the dataset doesn't contain the column names, so we don't actually know the features. Luckily, if we read the documentation, we see they tell us the column names are *name-of-code transform-type transform-size mflops time setup-time*. Using these features, we can pass them into the **load_data** function which will create the dataframe with column names.

In [ ]:
features = [:code_name, :transform_name, :size, :mflops, :time, :setup_time, :missing] # Extra missing added at end because the data contains an extra missing column
fftw_df = load_data("ibook-macosx.speed", colnames=features) # Modify the file name depending on what data you want to import
select!(fftw_df, Not(ncol(fftw_df))) # Removes the last column because there is an extra missing column

Row,code_name,transform_name,size,mflops,time,setup_time
,String15,String7,String15,Float64,Float64,Float64
1,arprec,dcif,4,27.09,1.47656e-6,9.5e-5
2,arprec,dcib,4,27.234,1.46875e-6,0.000113
3,arprec,dcif,8,66.783,1.79688e-6,9.8e-5
4,arprec,dcib,8,66.783,1.79688e-6,9.6e-5
5,arprec,dcif,16,124.88,2.5625e-6,0.000148
6,arprec,dcib,16,127.6,2.50781e-6,0.000149
7,arprec,dcif,32,196.17,4.07813e-6,0.000245
8,arprec,dcib,32,198.45,4.03125e-6,0.000243
9,arprec,dcif,64,253.88,7.5625e-6,0.000429


Fix the prompting

We can get the different transforms (the output) and the features (size). We will also make the simplifying assumption to care about 1-D transforms only. This will allow us to get rid of all rows with sizes that have more than 1 dimension.

In [ ]:
# Remove rows with higher dimensional fourier transforms
fftw_df = fftw_df[.!occursin.("x", fftw_df[!, :size]), :]
# Type cast those sizes to numbers to make computation easier later on
fftw_df[!, :size] = parse.(Int, fftw_df[!, :size])

# Get the inputs and outputs
transforms = unique(fftw_df[!, :code_name])
features = [:size]

1-element Vector{Symbol}:
 :size

Then, we can prompt the LLM to generate a heuristic that tells us the optimial transform type based on the size of the input we have.

In [ ]:
# Format heuristic prompt
function format_heuristic_prompt(data::DataFrame, algorithms, features)
    full_data = JSON3.write(data)

    prompt = """
    You are an expert in Fast Fourier Transform (FFT) algorithms. Below is a dataset with vector features ($features)
    and the best-performing FFT algorithm $algorithms for each input vector.

    Dataset: $full_data

    Using the dataset provided, please create a heuristic that I can use to find the optimal FFT algorithm
    for any combination of input vector features. The heuristic must cover all algorithms from the dataset: $algorithms.
    Ensure that each algorithm has a well-defined region in the feature space. Do not omit any algorithms or suggest only one algorithm.

    Please follow these steps:
    1. Analyze the dataset to understand how each algorithm performs under different feature conditions.
    2. Identify clear boundaries where one algorithm consistently outperforms others.
    3. Create a heuristic using numerical rules that covers all algorithms.
    4. Provide clear, readable guidelines for selecting the optimal algorithm for any given vector feature set.
    5. Ensure the heuristic covers all algorithms in the dataset, without suggesting any placeholder or alternative algorithms.

    ### Important Instructions for the Julia Code:

    - You must implement the heuristic as a Julia function using this exact signature:
        ```julia
        function choose_algorithm(; kwargs...)::String
        ```
    - Inside the function, access each vector feature using the `get` function:
        ```julia
    $features
        ```
    - Do **not** use named parameters in the function signature (e.g., `; length=...` is not allowed).
    - Use `if/elseif/else` logic to express the heuristic clearly.

    - You must return **only one** of the following FFT algorithms, as a string literal: $algorithms
    - Do **not** modify, rename, or add to these algorithm names.
    - Each `return` statement must use **only one** of the allowed strings exactly as written above.

    Your output should include:
    1. A plain-language explanation of the heuristic rules.
    2. The complete Julia code using the structure above.
    """

    return prompt
end

format_heuristic_prompt (generic function with 2 methods)

In [ ]:
# Create heuristics
function create_FFT_heuristics(data::DataFrame, file_name, algorithms, features)
    prompt = format_heuristic_prompt(data, algorithms, features)
    response = query_mistral(prompt)

    println(response)

    if isnothing(response)
        println("Failed to get a response from LLaMA.")
        return
    end

    code_match = match(r"```julia\n(.*?)```"s, response)
    if code_match !== nothing
        julia_code = strip(code_match.captures[1]) # Only keeps the julia code
        open(file_name, "w") do f
            write(f, julia_code)
        end
        println("Julia code extracted and saved to heuristic.jl")
    else
        println("No Julia code block found in the output.")
    end
end

create_FFT_heuristics (generic function with 1 method)

In [ ]:
create_FFT_heuristics(fftw_df[:, features], "heuristic.jl", transforms, features)

time=2025-04-11T18:48:08.598Z level=INFO source=server.go:105 msg="system memory" total="12.7 GiB" free="11.1 GiB" free_swap="0 B"
time=2025-04-11T18:48:08.598Z level=WARN source=ggml.go:152 msg="key not found" key=llama.vision.block_count default=0
time=2025-04-11T18:48:08.598Z level=WARN source=ggml.go:152 msg="key not found" key=llama.attention.key_length default=128
time=2025-04-11T18:48:08.598Z level=WARN source=ggml.go:152 msg="key not found" key=llama.attention.value_length default=128
time=2025-04-11T18:48:08.599Z level=INFO source=server.go:138 msg=offload library=cpu layers.requested=-1 layers.model=33 layers.offload=0 layers.split="" memory.available="[11.1 GiB]" memory.gpu_overhead="0 B" memory.required.full="5.5 GiB" memory.required.partial="0 B" memory.required.kv="1.0 GiB" memory.required.allocations="[5.5 GiB]" memory.weights.total="3.8 GiB" memory.weights.repeating="3.7 GiB" memory.weights.nonrepeating="105.0 MiB" memory.graph.full="560.0 MiB" memory.graph.partial="585

[GIN] 2025/04/11 - 19:14:50 | 200 |        26m42s |       127.0.0.1 | POST     "/api/chat"
After analyzing the dataset, we can identify several trends that might help us create a heuristic to select the optimal FFT algorithm based on input vector features. Here's a breakdown of how each algorithm performs under various conditions:

1. `arprec`: This algorithm seems to work well when the number of elements in the input vector is relatively small (fewer than 1024).
2. `bloodworth` and `bloodworth-fht`: These algorithms perform better for larger input vectors (greater than 1024 elements).
3. `cross`, `cwplib`, `fftreal`, `fftw3`, `fftw3-r2r`, `green`, `jmfftc`, `krukar`, `mixfft`, `monnier`, `newsplit`, `numutils`, `ooura-4g`, `ooura-8g`, `ooura-sg`, `qft`, `ransom`, `scimark2c`, `valkenburg`, `vbigdsp`, and `vdsp`: These algorithms are optimized for larger input vectors (greater than 1024 elements). However, they have varying performance characteristics that make them more suitable for s

[ Info: Tokens: 2630 in 1606.0 seconds
